In [ ]:
"""
I jst simply use the code from other's shared notebook:
https://www.kaggle.com/chixujohnny/try-to-use-nn-baseline/notebook
And I modified the classification problem to a regression problem. And simply return 1 when predicted resp
is positive, return 0 when resp is negative. I only spend a few days working on this competetion.
"""

In [ ]:
"""
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"""

import os

import tensorflow as tf
from tensorflow.keras import models,layers,losses,metrics,callbacks 
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)


In [ ]:
train = pd.read_csv("../input/jane-street-market-prediction/train.csv")

In [ ]:
train = train.query('date > 85').reset_index(drop = True)

In [ ]:
train.head()

In [ ]:
features = []
for item in train.columns:
#     if 'feature' in item or 'weight' in item:
    if 'feature' in item:
        features.append(item)
#print(features)

In [ ]:
features_mean = train.loc[:, features].mean()

In [ ]:
train.fillna(train.mean(), inplace=True)

In [ ]:
resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp']
# train
X_train = train.loc[:, features].values
y_train = np.stack([train[c] for c in resp_cols]).T
print(X_train.shape, y_train.shape)
del train

In [ ]:
"""
print('Create a easy NN model...')

HIDDEN_LAYER = [160, 160, 160]
#HIDDEN_LAYER = [4, 4, 4] #for fast debug test
TARGET_NUM = 5   # 优化那5个resp

input = tf.keras.layers.Input(shape=(X_train.shape[1], ))

x = tf.keras.layers.BatchNormalization()(input)
x = tf.keras.layers.Dropout(0.2)(x)
for units in HIDDEN_LAYER:
    x = tf.keras.layers.Dense(units)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)  # 除了ReLU还可以试试别的，后面可以做一个多模型融合
    x = tf.keras.layers.Dropout(0.2)(x)
output = tf.keras.layers.Dense(TARGET_NUM)(x)

#output = tf.keras.layers.Activation("sigmoid")(x)

model = tf.keras.models.Model(inputs=input, outputs=output)
model.compile(
    optimizer = tf.optimizers.Adam(learning_rate=1e-3),
    #metrics   = tf.keras.metrics.AUC(name="AUC"),
    metrics   = tf.keras.metrics.MeanSquaredError(name='mean_squared_error'),
    #loss      = tf.keras.losses.BinaryCrossentropy(label_smoothing=1e-2),
    loss      = "MSE",
    
)

print('Done!')
"""

In [ ]:
HIDDEN_LAYER_1 = [256, 256]
HIDDEN_LAYER_2 = [160, 160, 160]
HIDDEN_LAYER_3 = [128, 128, 128, 128]
TARGET_NUM = 5   # 优化那5个resp

input = tf.keras.layers.Input(shape=(X_train.shape[1], ))

#part_1
x1 = tf.keras.layers.BatchNormalization()(input)
x1 = tf.keras.layers.Dropout(0.25)(x1)
for units in HIDDEN_LAYER_1:
    x1 = tf.keras.layers.Dense(units)(x1)
    x1 = tf.keras.layers.BatchNormalization()(x1)
    x1 = tf.keras.layers.Activation(tf.keras.activations.swish)(x1)
    x1 = tf.keras.layers.Dropout(0.25)(x1)

# part_2
x2 = tf.keras.layers.BatchNormalization()(input)
x2 = tf.keras.layers.Dropout(0.25)(x2)
for units in HIDDEN_LAYER_2:
    x2 = tf.keras.layers.Dense(units)(x2)
    x2 = tf.keras.layers.BatchNormalization()(x2)
    x2 = tf.keras.layers.Activation(tf.keras.activations.swish)(x2)
    x2 = tf.keras.layers.Dropout(0.25)(x2)

# part_3
x3 = tf.keras.layers.BatchNormalization()(input)
x3 = tf.keras.layers.Dropout(0.25)(x3)
for units in HIDDEN_LAYER_3:
    x3 = tf.keras.layers.Dense(units)(x3)
    x3 = tf.keras.layers.BatchNormalization()(x3)
    x3 = tf.keras.layers.Activation(tf.keras.activations.swish)(x3)
    x3 = tf.keras.layers.Dropout(0.25)(x3)

x = tf.keras.layers.concatenate([x1, x2, x3])
output = tf.keras.layers.Dense(TARGET_NUM)(x)

#output = tf.keras.layers.Activation("sigmoid")(x)

model = tf.keras.models.Model(inputs=input, outputs=output)
model.compile(
    optimizer = tf.optimizers.Adam(learning_rate=1e-3),
    #metrics   = tf.keras.metrics.AUC(name="AUC"),
    metrics   = tf.keras.metrics.MeanSquaredError(name='mean_squared_error'),
    #loss      = tf.keras.losses.BinaryCrossentropy(label_smoothing=1e-2),
    loss      = "MSE",
    
)

from keras.utils import plot_model
display(plot_model(model))

print('Done!')

In [ ]:
print('Train NN...')

history = model.fit(
    x = X_train, 
    y = y_train, 
    epochs=200,
    #epochs=1, # for debug
    batch_size=4096, 
#     validation_data=(X_valid, y_valid),
#     callbacks = EarlyStopping(monitor='roc_auc', patience=20, verbose=2, mode='max')
)
models = []
models.append(model)

print('Done!')

In [ ]:
del X_train, y_train

In [ ]:

# save model

model.save('./model', save_format="tf")
print('export saved model.')



In [ ]:
model = tf.keras.models.load_model('./model')

In [ ]:
THRESHOLD = 0


import janestreet
from tqdm import tqdm
#janestreet.make_env.__called__ = False
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set

print('predicting...')

for (test_df, pred_df) in tqdm(env.iter_test()):
    global pred
    if test_df['weight'].item() > 0:    
        X_test = test_df.loc[:, features].values
        if np.isnan(X_test.sum()):  # 用numpy填充，速度会比较快
            X_test = np.nan_to_num(X_test) + np.isnan(X_test) * features_mean.values
        pred = model(X_test, training = False).numpy()
        pred = np.mean(pred)
        #pred = pred[0][4]

        pred_df.action = np.where(pred > THRESHOLD, 1, 0).astype(int)
    else:
        pred_df.action = 0
    env.predict(pred_df)
